In [1]:
import csv
import numpy as np
import pandas as pd
import json
from functools import reduce

In [5]:
# data = pd.read_csv("TestData/MockData2-goodNames-normalized.csv") 
# data = pd.read_csv("TestData/COVID.csv")
data = pd.read_csv("TestData/1_ToyDataset-All.csv") 

In [6]:
data

,State,SourceID,SourceNodeColor,SourceNodeSize,EdgeColor,EdgeWeight,TargetID,TargetNodeSize,PhosphorylationSiteID,TargetNodeColor
0,TimeStamp0,Source1,-0.820,1.200,-1.800,3.38,Target1,1.16,SiteTarget1-1,-1.76
1,TimeStamp0,Source1,-0.820,1.200,-1.800,3.38,Target1,1.16,SiteTarget1-2,0.88
2,TimeStamp0,Source1,-0.820,1.200,-1.800,3.38,Target1,1.16,SiteTarget1-3,0.67
3,TimeStamp0,Source1,-0.820,1.200,-1.800,3.38,Target1,1.16,SiteTarget1-4,-0.71
4,TimeStamp0,Source1,-0.820,1.200,-0.140,0.60,Target2,0.77,SiteTarget2-1,-1.35
5,TimeStamp0,Source1,-0.820,1.200,2.946,0.55,Target3,2.31,SiteTarget3-1,3.31
6,TimeStamp0,Source1,-0.820,1.200,2.946,0.55,Target3,2.31,SiteTarget3-2,-1.72
7,TimeStamp0,Source1,-0.820,1.200,2.946,0.55,Target3,2.31,SiteTarget3-3,-1.18
8,TimeStamp0,Source1,-0.820,1.200,2.946,0.55,Target3,2.31,SiteTarget3-4,-2.72
9,TimeStamp0,Source1,-0.820,1.200,2.480,3.14,Target4,2.12,SiteTarget4-1,3.28


In [7]:
def normalize(values, actual_bounds, desired_bounds):
    final = []
    for i in values:
        final.append(desired_bounds[0] + (i - actual_bounds[0]) * (desired_bounds[1] - desired_bounds[0]) / (actual_bounds[1] - actual_bounds[0]))
    return final

In [8]:
def attribute_dict (dataframe, col_name_source_or_target, col_name_attribute_of_interest):
    dictionary = {}
    
    min_val = min(dataframe[col_name_attribute_of_interest].values)
    max_val = max(dataframe[col_name_attribute_of_interest].values)
    actual_bounds = (min_val, max_val)
    desired_bounds = (0, 1)
    
    for i in range(len(dataframe)):
        if col_name_attribute_of_interest in dataframe.columns:
            value = dataframe[col_name_attribute_of_interest][i]
            dictionary[dataframe[col_name_source_or_target][i]] = normalize([value], actual_bounds, desired_bounds)[0]
        else:
            dictionary[dataframe[col_name_source_or_target][i]] = None
            
    return dictionary

In [9]:
source_att_1 = attribute_dict(data, "SourceID", "SourceNodeColor")
source_att_1

{'Source1': 0.14634146341463417,
 'Source2': 1.0,
 'Source3': 0.7392120075046904,
 'Source4': 0.0,
 'Source5': 0.7450281425891182,
 'Source6': 0.8405253283302064}

In [19]:
def partitions_dict (dataframe, col_name_target, col_name_partitions, col_name_attribute_of_interest):
    partitions = {}
    rep = {}

    target = dataframe[col_name_target]
    val = dataframe[col_name_attribute_of_interest]

    if col_name_partitions in dataframe.columns:
        site = dataframe[col_name_partitions]
        for i in range(len(dataframe)):
            if target[i] not in partitions:
                partitions[target[i]] = [{site[i]: val[i]}]
                rep[target[i]] = [site[i]]

            else:
                if site[i] not in rep[target[i]]:
                    partitions[target[i]] += [{site[i]: val[i]}]
                    rep[target[i]] += [site[i]]
    
    return partitions    

In [11]:
def links_info (dataframe, col_name_source, col_name_target):
    links = []

    for i in range(len(data)):
        if [dataframe[col_name_source][i], dataframe[col_name_target][i]] not in links:
            links.append([dataframe[col_name_source][i], dataframe[col_name_target][i]])
    return links

In [14]:
source_att_1 = attribute_dict(data, "SourceID", "SourceNodeColor")
source_att_2 = attribute_dict(data, "SourceID", "SourceNodeSize")
target_att_1 = attribute_dict(data, "TargetID", "TargetNodeColor")
target_att_2 = attribute_dict(data, "TargetID", "TargetNodeSize")
edge_att_1 = attribute_dict(data, "TargetID", "EdgeColor")
edge_att_2 = attribute_dict(data, "TargetID", "EdgeWeight")

partitions = partitions_dict(data, "TargetID", "PhosphorylationSiteID", "TargetNodeColor")
links = links_info(data, "SourceID", "TargetID")

In [18]:
partitions

{'Target1': [{'SiteTarget1-1': -1.76},
  {'SiteTarget1-2': 0.88},
  {'SiteTarget1-3': 0.67},
  {'SiteTarget1-4': -0.71}],
 'Target2': [{'SiteTarget2-1': -1.35}],
 'Target3': [{'SiteTarget3-1': 3.31},
  {'SiteTarget3-2': -1.72},
  {'SiteTarget3-3': -1.18},
  {'SiteTarget3-4': -2.72}],
 'Target4': [{'SiteTarget4-1': 3.28}],
 'Target5': [{'SiteTarget5-1': -2.52}],
 'Target6': [{'SiteTarget6-1': 2.72}],
 'Target7': [{'SiteTarget7-1': 2.2}],
 'Target8': [{'SiteTarget8-1': 1.1}],
 'Target9': [{'SiteTarget9-1': -1.97}, {'SiteTarget9-2': 1.74}],
 'Target10': [{'SiteTarget10-1': -0.93}],
 'Target11': [{'SiteTarget11-1': -1.53}],
 'Target12': [{'SiteTarget12-1': -1.53}],
 'Target13': [{'SiteTarget13-1': 1.21}],
 'Target14': [{'SiteTarget14-1': -2.68}],
 'Target15': [{'SiteTarget15-1': 3.34}],
 'Target16': [{'SiteTarget16-1': -3.26}],
 'Target17': [{'SiteTarget17-1': -0.46},
  {'SiteTarget17-2': 1.35},
  {'SiteTarget17-3': -3.58},
  {'SiteTarget17-4': -0.67},
  {'SiteTarget17-5': -0.38},
  {'Site

In [20]:
edge_att_1

{'Target1': 0.22206703910614523,
 'Target2': 0.5111731843575419,
 'Target3': 0.5391061452513967,
 'Target4': 0.877094972067039,
 'Target5': 0.5754189944134078,
 'Target6': 0.11871508379888263,
 'Target7': 0.03351955307262566,
 'Target8': 0.803072625698324,
 'Target9': 0.12430167597765358,
 'Target10': 0.2108938547486033,
 'Target11': 0.6606145251396647,
 'Target12': 0.9441340782122905,
 'Target13': 0.3072625698324022,
 'Target14': 0.04050279329608939,
 'Target15': 0.7807262569832402,
 'Target16': 0.0,
 'Target17': 0.8659217877094971,
 'Target18': 0.7625698324022346,
 'Target19': 0.4371508379888268,
 'Target20': 0.20530726256983237,
 'Target21': 0.8072625698324021,
 'Target22': 0.8072625698324021,
 'Target23': 1.0,
 'Target24': 0.6648044692737429,
 'Target25': 0.9790502793296089,
 'Target26': 0.9790502793296089}

In [21]:
testing_a_null_column = partitions_dict(data, "TargetID", "hello", "TargetNodeColor")
testing_a_null_column

{}

In [22]:
dictionary = {"nodes": [], "links": []}

source = data["SourceID"].unique()
target = data["TargetID"].unique()

for s in source:
    dictionary['nodes'] += [{'id': s, "group": 1, "source_attr_1": source_att_1[s], "source_attr_2": source_att_1[s]}]
    
for t in target:
    dictionary['nodes'] += [{'id': t, "group": 2, "target_attr_2": target_att_2[t], "positions": partitions[t]}]
    
for s, t in links:
    dictionary['links'] += [{"source": s, "target": t, "edge_attr_1": edge_att_1[t], "edge_attr_2": edge_att_2[t]}]

In [32]:
with open('mockdata.json', 'w') as fp:
    json.dump(dictionary, fp)

In [23]:
dictionary

{'nodes': [{'id': 'Source1',
   'group': 1,
   'source_attr_1': 0.14634146341463417,
   'source_attr_2': 0.14634146341463417},
  {'id': 'Source2', 'group': 1, 'source_attr_1': 1.0, 'source_attr_2': 1.0},
  {'id': 'Source3',
   'group': 1,
   'source_attr_1': 0.7392120075046904,
   'source_attr_2': 0.7392120075046904},
  {'id': 'Source4', 'group': 1, 'source_attr_1': 0.0, 'source_attr_2': 0.0},
  {'id': 'Source5',
   'group': 1,
   'source_attr_1': 0.7450281425891182,
   'source_attr_2': 0.7450281425891182},
  {'id': 'Source6',
   'group': 1,
   'source_attr_1': 0.8405253283302064,
   'source_attr_2': 0.8405253283302064},
  {'id': 'Target1',
   'group': 2,
   'target_attr_2': 0.29639175257731953,
   'positions': [{'SiteTarget1-1': -1.76},
    {'SiteTarget1-2': 0.88},
    {'SiteTarget1-3': 0.67},
    {'SiteTarget1-4': -0.71}]},
  {'id': 'Target2',
   'group': 2,
   'target_attr_2': 0.1958762886597938,
   'positions': [{'SiteTarget2-1': -1.35}]},
  {'id': 'Target3',
   'group': 2,
   'tar